# Up and Running with Tensor Flow

In [1]:
import numpy as np
import numpy.random as rnd
import tensorflow as tf

from datetime import datetime
from IPython.display import clear_output, display, HTML, Image
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
# Can reset graph at any time (instead of rebooting jupyter) with:
#tf.reset_default_graph()

In [3]:
x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
f = x * x * y + y + 2

s = tf.Session()
s.run(x.initializer)
s.run(y.initializer)
res = s.run(f)
print(res)
s.close()

42


In [4]:
# Or
with tf.Session() as s:
    x.initializer.run()
    y.initializer.run()
    res = f.eval()
    
    print(res)

42


In [5]:
# Or init all vars
init = tf.global_variables_initializer() # preps an init node

with tf.Session() as s:
    init.run() # actual var initialization
    res = f.eval()
    print(res)

42


### Managing Graphs

In [6]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

# Make new graph and temporarily set is as default
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
True
False


### Lifecycle of a Node Value

In [7]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as s:
    print(y.eval())
    print(z.eval())

10
15


### Linear Regression in TF

In [8]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

housing_data_plus_bias[1:5, :]

array([[  1.00000000e+00,   8.30140000e+00,   2.10000000e+01,
          6.23813708e+00,   9.71880492e-01,   2.40100000e+03,
          2.10984183e+00,   3.78600000e+01,  -1.22220000e+02],
       [  1.00000000e+00,   7.25740000e+00,   5.20000000e+01,
          8.28813559e+00,   1.07344633e+00,   4.96000000e+02,
          2.80225989e+00,   3.78500000e+01,  -1.22240000e+02],
       [  1.00000000e+00,   5.64310000e+00,   5.20000000e+01,
          5.81735160e+00,   1.07305936e+00,   5.58000000e+02,
          2.54794521e+00,   3.78500000e+01,  -1.22250000e+02],
       [  1.00000000e+00,   3.84620000e+00,   5.20000000e+01,
          6.28185328e+00,   1.08108108e+00,   5.65000000e+02,
          2.18146718e+00,   3.78500000e+01,  -1.22250000e+02]])

In [9]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(
    housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
XT = tf.transpose(X)
# Normal equation: Theta = (X.TX)^-1X.Ty
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as s:
    theta_value = theta.eval()
    
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

### Implementing Gradient Descent

In [10]:
# First scale features
scaled_housing = StandardScaler().fit_transform(housing.data)
scaled_housing_bias = np.c_[np.ones((m, 1)), scaled_housing]

# Sklearn Regression model for comparison:
lm = LinearRegression()
lm.fit(scaled_housing, housing.target.reshape(-1, 1))

#np.r_?

print(np.r_[lm.intercept_.reshape(-1, 1), lm.coef_.T])

[[ 2.06855817]
 [ 0.8296193 ]
 [ 0.11875165]
 [-0.26552688]
 [ 0.30569623]
 [-0.004503  ]
 [-0.03932627]
 [-0.89988565]
 [-0.870541  ]]


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


#### Manually Computing the Gradients

In [11]:
n_epochs = 4000 # iterations of (batch) GD
eta = 0.01      # learning rate

tf.reset_default_graph()
X = tf.constant(scaled_housing_bias, dtype = tf.float32, name = 'X')
y = tf.constant(
    housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1.), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
# assign() updates theta
training_op = tf.assign(theta, theta - eta * gradients)

init = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init)
    for epoch in range(n_epochs):
        if epoch % 400 == 0:
            print('Epoch %4d\tMSE = %.4f' %(epoch, mse.eval()))
        s.run(training_op)
        
    best_theta = theta.eval()
    print(best_theta) # Note: vals differ from above b/c of scaling

Epoch    0	MSE = 6.0242
Epoch  400	MSE = 0.6259
Epoch  800	MSE = 0.5638
Epoch 1200	MSE = 0.5408
Epoch 1600	MSE = 0.5315
Epoch 2000	MSE = 0.5276
Epoch 2400	MSE = 0.5259
Epoch 2800	MSE = 0.5250
Epoch 3200	MSE = 0.5247
Epoch 3600	MSE = 0.5245
[[ 2.06855249]
 [ 0.83938348]
 [ 0.1207923 ]
 [-0.28358203]
 [ 0.32045361]
 [-0.00388739]
 [-0.03970707]
 [-0.8758412 ]
 [-0.84761393]]


#### Using TF's `autodiff`

In [12]:
n_epochs = 4000 # iterations of (batch) GD
eta = 0.01      # learning rate

tf.reset_default_graph()
X = tf.constant(scaled_housing_bias, dtype = tf.float32, name = 'X')
y = tf.constant(
    housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1.), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = tf.gradients(mse, [theta])[0]
# assign() updates theta
training_op = tf.assign(theta, theta - eta * gradients)

init = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init)
    for epoch in range(n_epochs):
        if epoch % 400 == 0:
            print('Epoch %4d\tMSE = %.4f' %(epoch, mse.eval()))
        s.run(training_op)
        
    best_theta = theta.eval()
    print(best_theta) # Note: Converges to same degree of accuracy sooner

Epoch    0	MSE = 13.5263
Epoch  400	MSE = 0.6515
Epoch  800	MSE = 0.5590
Epoch 1200	MSE = 0.5338
Epoch 1600	MSE = 0.5270
Epoch 2000	MSE = 0.5251
Epoch 2400	MSE = 0.5245
Epoch 2800	MSE = 0.5244
Epoch 3200	MSE = 0.5243
Epoch 3600	MSE = 0.5243
[[ 2.06855249]
 [ 0.82750708]
 [ 0.11875076]
 [-0.26079711]
 [ 0.30146012]
 [-0.00447591]
 [-0.0392811 ]
 [-0.90187538]
 [-0.87224895]]


#### Using an Optimizer

In [13]:
n_epochs = 4000 # iterations of (batch) GD
eta = 0.01      # learning rate

tf.reset_default_graph()
X = tf.constant(scaled_housing_bias, dtype = tf.float32, name = 'X')
y = tf.constant(
    housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1.), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = eta)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init)
    for epoch in range(n_epochs):
        if epoch % 400 == 0:
            print('Epoch %4d\tMSE = %.4f' %(epoch, mse.eval()))
        s.run(training_op)
        
    best_theta = theta.eval()
    print(best_theta) 

Epoch    0	MSE = 11.6607
Epoch  400	MSE = 0.5677
Epoch  800	MSE = 0.5435
Epoch 1200	MSE = 0.5331
Epoch 1600	MSE = 0.5284
Epoch 2000	MSE = 0.5263
Epoch 2400	MSE = 0.5253
Epoch 2800	MSE = 0.5248
Epoch 3200	MSE = 0.5245
Epoch 3600	MSE = 0.5244
[[ 2.06855249]
 [ 0.83746254]
 [ 0.12030237]
 [-0.28019547]
 [ 0.31775993]
 [-0.00404068]
 [-0.03962468]
 [-0.88121635]
 [-0.85277724]]


### Feeding Data to the Training Algorithm
Modify code to run as mini-batch GD

In [14]:
# Using placeholders
# 'None' allows any no. of rows:
A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5

with tf.Session() as s:
    B1 = B.eval(feed_dict = { A: [[1, 2, 3]] })
    B2 = B.eval(feed_dict = { A: [[4, 5, 6], 
                                  [7, 8, 9]] })
    
print(B1)
print(B2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [15]:
def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size = batch_size)
    X_batch = scaled_housing_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [16]:
tf.reset_default_graph()

n_epochs = 40 # iterations of (batch) GD
eta = 0.01      # learning rate

m, n = housing.data.shape
X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1., seed = 42), 
                    name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = eta)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))


with tf.Session() as s:
    s.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            s.run(training_op, feed_dict = { X: X_batch, y: y_batch })
        
    best_theta = theta.eval()
    
print('Best theta:\n', best_theta) 

Best theta:
 [[ 2.06604695]
 [ 0.84577423]
 [ 0.11162421]
 [-0.27690282]
 [ 0.28115126]
 [-0.02072373]
 [-0.06640353]
 [-0.8829757 ]
 [-0.85721791]]


### Saving and Restoring Models

In [17]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_bias, dtype = tf.float32, name = 'X')
y = tf.constant(
    housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1., seed = 42), 
                    name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = eta)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [18]:
with tf.Session() as s:
    s.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch: %3d\tMSE: %.4f' %(epoch, mse.eval()))
            save_path = saver.save(s, '/tmp/ch09_mod.ckpt')
        s.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(s, 'ch09_final_mod.ckpt')
    
print('Best theta:\n', best_theta)

Epoch:   0	MSE: 2.7544
Epoch: 100	MSE: 0.6322
Epoch: 200	MSE: 0.5728
Epoch: 300	MSE: 0.5585
Epoch: 400	MSE: 0.5491
Epoch: 500	MSE: 0.5423
Epoch: 600	MSE: 0.5374
Epoch: 700	MSE: 0.5338
Epoch: 800	MSE: 0.5312
Epoch: 900	MSE: 0.5294
Best theta:
 [[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


In [19]:
!ls | grep 'mod'

ch09_final_mod.ckpt.data-00000-of-00001
ch09_final_mod.ckpt.index
ch09_final_mod.ckpt.meta


In [20]:
# Load a saved model
with tf.Session() as s:
    saver.restore(s, '/tmp/ch09_mod.ckpt')
    s.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch: %3d\tMSE: %.4f' %(epoch, mse.eval()))
            save_path = saver.save(s, '/tmp/ch09_mod.ckpt')
        s.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(s, 'ch09_final_mod.ckpt')
    
print('Best theta:\n', best_theta)

INFO:tensorflow:Restoring parameters from /tmp/ch09_mod.ckpt
Epoch:   0	MSE: 2.7544
Epoch: 100	MSE: 0.6322
Epoch: 200	MSE: 0.5728
Epoch: 300	MSE: 0.5585
Epoch: 400	MSE: 0.5491
Epoch: 500	MSE: 0.5423
Epoch: 600	MSE: 0.5374
Epoch: 700	MSE: 0.5338
Epoch: 800	MSE: 0.5312
Epoch: 900	MSE: 0.5294
Best theta:
 [[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


### Visualizing the Graph in Jupyter

In [21]:
def strip_consts(graph_def, max_const_size = 32):
    '''Strip large constant values from graph_def'''
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b'<stripped %d bytes>'%size
    return strip_def

def show_graph(graph_def, max_const_size = 32):
    '''Visualize TF graph'''
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size = max_const_size)
    code = '''
      <script>
        function load() {{
          document.getElementById('{id}').pbtxt = {data};
        }}
      </script>
      <link 
         rel="import" 
         href="https://tensorboard.appspot.com/tf-graph-basic.build.html"
         onload=load() />
      <div style="height:600px">
        <tf-graph-basic id="{id}"><t/tf-graph-basic>
      </div>
    '''.format(data = repr(str(strip_def)), 
               id = 'graph' + str(np.random.rand()))
    iframe = '''
      <iframe seamless style="width:1200px;height:620px;border:0" 
              srcdoc="{}">
      </iframe>
    '''.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [22]:
show_graph(tf.get_default_graph())

### Visualizing with TensorBoard

In [23]:
tf.reset_default_graph()
now = datetime.utcnow().strftime('%Y%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

In [24]:
eta = 0.01

X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1., seed = 42), 
                    name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = eta)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [25]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as s:
    s.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(
                    feed_dict = { X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                summary_writer.add_summary(summary_str, step)
            s.run(training_op, feed_dict = { X: X_batch, y: y_batch})
            
    best_theta = theta.eval()
    
summary_writer.flush()
summary_writer.close()
print('Best theta:\n', best_theta)

Best theta:
 [[ 2.07001591]
 [ 0.82045609]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.31134021]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.87950081]]


In [26]:
!ls -l tf_logs/

total 0
drwxr-xr-x  3 dsatterthwaite  staff  102 May  2 08:55 run-201702155537
drwxr-xr-x  5 dsatterthwaite  staff  170 Apr 26 15:25 run-201726213820


From the command line run:

`> source ~/tensorflow/bin/activate`

`> tensorboard --logdir tf_logs/`

Open in localhost:6006

## Name Scopes

In [27]:
tf.reset_default_graph()

now = datetime.utcnow().strftime('%Y%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

In [28]:
X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1., 1., seed = 42), 
                    name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')

# Wrap 'error' and 'mse' within the 'loss' name scope
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate = eta)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [29]:
n_epochs = 20
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as s:
    s.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(
                    feed_dict = { X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                summary_writer.add_summary(summary_str, step)
            s.run(training_op, feed_dict = { X: X_batch, y: y_batch})
            
    best_theta = theta.eval()
    
summary_writer.flush()
summary_writer.close()
print('Best theta:\n', best_theta)

Best theta:
 [[ 2.06562495]
 [ 0.82996982]
 [ 0.11778412]
 [-0.27003285]
 [ 0.29147986]
 [-0.00638249]
 [-0.05427555]
 [-0.89488846]
 [-0.8678754 ]]


In [30]:
print(error.op.name)

loss/sub


In [31]:
print(mse.op.name)

loss/mse


### Modularity

In [32]:
def relu(X):
    with tf.name_scope('relu'):
        W_shape = (int(X.get_shape()[1]), 1)
        W = tf.Variable(tf.random_normal(W_shape), name = 'weights')
        b = tf.Variable(0.0, name = 'bias')
        z = tf.add(tf.matmul(X, W), b, name = 'z')
        return tf.maximum(z, 0., name = 'relu')

In [33]:
n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X) for i in range(5)]
# add_n() computes the sum of a list of tensors
outputs = tf.add_n(relus, name = 'output')

## Sharing Variables

In [35]:
tf.reset_default_graph()

def relu(X, threshold):
    with tf.name_scope('relu'):
        W_shape = (int(X.get_shape()[1]), 1)
        W = tf.Variable(tf.random_normal(W_shape), name = 'weights')
        b = tf.Variable(0.0, name = 'bias')
        linear = tf.add(tf.matmul(X, W), b, name = 'linear')
        return tf.maximum(linear, threshold, name = 'max')
    
threshold = tf.Variable(0.0, name = 'threshold')
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name = 'output')

In [38]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope('relu'):
        if not hasattr(relu, 'threshold'):
            relu.threshold = tf.Variable(0.0, name = 'threshold')
        W_shape = (int(X.get_shape()[1]), 1)
        W = tf.Variable(tf.random_normal(W_shape), name = 'weights')
        b = tf.Variable(0.0, name = 'bias')
        linear = tf.add(tf.matmul(X, W), b, name = 'linear')
        return tf.maximum(linear, relu.threshold, name = 'max')

X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name = 'output')

In [39]:
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope('relu', reuse = True):
        threshold = tf.get_varibale(
            'threshold', 
            shape = (), 
            initializer = tf.constant.initializer(0.0))
        W_shape = (int(X.get_shape()[1]), 1)
        W = tf.Variable(tf.random_normal(W_shape), name = 'weights')
        b = tf.Variable(0.0, name = 'bias')
        linear = tf.add(tf.matmul(X, W), b, name = 'linear')
        return tf.maximum(linear, threshold, name = 'max')

In [40]:
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope('relu'):
        threshold = tf.get_variable(
            'threshold', 
            shape = (), 
            initializer = tf.constant_initializer(0.0))
        W_shape = int(X.get_shape()[1]), 1
        W = tf.Variable(tf.random_normal(W_shape), name = 'weights')
        b = tf.Variable(0.0, name = 'bias')
        linear = tf.add(tf.matmul(X, W), b, name = 'linear')
        return tf.maximum(linear, threshold, name = 'max')

X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')

with tf.variable_scope('', default_name = '') as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
    
output = tf.add_n(relus, name = 'output')

#summary_writer = tf.summary.FileWriter(
#    'logs/relu8', tf.get_default_graph())
#summary_writer.close()